In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.optimize import differential_evolution


In [ ]:
df = pd.read_csv("ECTA 2023 Dataset.csv")
features = ['Relative Humidity', 'Wind Speed', 'Visibility', 'Pressure', 'Wind_Chill', 'Dewpoint Temp']
target = 'Temperature'

In [ ]:
df = df.dropna()
X = df[features].values
y = df[target].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def build_ann(lr, neurons1, neurons2):
    model = keras.Sequential([
        keras.layers.Dense(neurons1, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(neurons2, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model


In [ ]:
HP_BOUNDS = [(0.0001, 0.1), (16, 128), (16, 128), (8, 64)]

eval_counter = 0  # Keep track of number of evaluations

def evaluate(individual):
    global eval_counter
    eval_counter += 1
    lr, neurons1, neurons2, batch_size = individual
    neurons1, neurons2, batch_size = int(neurons1), int(neurons2), int(batch_size)
    print(f"\n[{eval_counter}] Evaluating: LR={lr:.5f}, Neurons1={neurons1}, Neurons2={neurons2}, Batch Size={batch_size}")
    
    model = build_ann(lr, neurons1, neurons2)
    history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1)
    
    loss, mae = model.evaluate(X_test, y_test, verbose=1)
    print(f"[{eval_counter}] MAE: {mae:.4f}")
    
    return mae

In [ ]:
HP_BOUNDS = [(0.0001, 0.1), (16, 128), (16, 128), (8, 64)]

eval_counter = 0  # Track number of model evaluations

def evaluate(individual):
    global eval_counter
    eval_counter += 1
    lr, neurons1, neurons2, batch_size = individual
    neurons1, neurons2, batch_size = int(neurons1), int(neurons2), int(batch_size)
    print(f"\n[{eval_counter}] Evaluating: LR={lr:.5f}, Neurons1={neurons1}, Neurons2={neurons2}, Batch Size={batch_size}")

    model = build_ann(lr, neurons1, neurons2)
    history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1)  # ✅ Show progress

    loss, mae = model.evaluate(X_test, y_test, verbose=1)
    print(f"[{eval_counter}] MAE: {mae:.4f}")
    return mae

In [ ]:
result = differential_evolution(
    evaluate,
    HP_BOUNDS,
    strategy='best1bin',
    popsize=5,
    mutation=0.5,
    recombination=0.7,
    maxiter=1,             
    polish=False,         
    disp=True,              
    workers=1
)


In [ ]:
best_hyperparams = result.x
print(f"Best Hyperparameters: {best_hyperparams}")

best_hyperparams = result.x
print(f"\nBest Hyperparameters: {best_hyperparams}")


best_lr, best_neurons1, best_neurons2, best_batch_size = best_hyperparams
final_model = build_ann(best_lr, int(best_neurons1), int(best_neurons2))
final_model.fit(X_train, y_train, epochs=50, batch_size=int(best_batch_size), 
                validation_data=(X_test, y_test), verbose=1)

In [ ]:
loss, mae = final_model.evaluate(X_test, y_test, verbose=1)
print(f"Final Model Test MAE: {mae:.4f}")
y_pred = final_model.predict(X_test, verbose=1)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test[:10], label='Actual Temperature', color='blue', marker='o')
plt.plot(y_pred[:10], label='Predicted Temperature', color='red', linestyle='dashed', marker='x')
plt.legend()
plt.xlabel("Sample Index")
plt.ylabel("Temperature")
plt.title("Actual vs. Predicted Temperature (10 samples)")
plt.show()

In [ ]:
future_X = np.tile(np.mean(X_test, axis=0), (24, 1))
future_y_pred = final_model.predict(future_X, verbose=1)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(future_y_pred, marker='o', linestyle='dashed', color='green', label='Predicted Temperature')
plt.xlabel("Hours Ahead")
plt.ylabel("Temperature")
plt.title("Next 24-Hour Weather Prediction")
plt.legend()
plt.show()